# 메모리 한계 넘어가는 큰 이미지 분할 및 결합 구현

In [1]:
import numpy as np
import cv2
import time
import os
from enum import Enum
from multipledispatch import dispatch
import mrs3 as mr
import interpolation as inter

%load_ext autoreload
%autoreload 2

lenna_path = 'Lenna_(test_image).png'

def imshow(img, winname='test'):
    cv2.imshow(winname, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [10]:
img = cv2.imread('sample-images-png/1920x1080.png')
h, w = img.shape[:2]

In [ ]:
DIV_UNIT = 530 # 12g + 16g 기준
OVERLAP_LENGTH = 30 # at least

In [ ]:
"""
at most DIV_UNIT 으로 끊어서

1. 가능한 큰 조각씩 끊고 많이 겹쳐진 후에 블렌딩을 하는게 좋을지
2. 최소한의 크기로 끊고 겹치는 부분이 작은게 좋을지
"""



1. 가능한 큰 조각씩 끊고 많이 겹쳐진 후에 블렌딩을 하는게 좋을지

In [ ]:
"""

DIV + (DIV-OVER)*(n-1) = n * (DIV-OVER) + OVER

DIV * n - OVER * (n-1)

(DIV - OVER) * 2 + (DIV - 2*OVER) * (n-2) + (n-1) * OVER = n DIV - (n-1) OVER

DIV = DIV_UNIT
OVER >= OVERLAP_LENGTH

argmax{n}_{width = n * (div-over) + over, over>=len}

"""

In [ ]:
cnt = 0

2. 최소한의 크기로 끊고 겹치는 부분이 작은게 좋을지

In [ ]:
PIXELS_LIMIT = 300000
OVERLAP_HALF_LENGTH = 50

upscaled_fraction_num = 0
def upscale_img(img, scaler):
    """
    img: 이미지 ndarray
    scaler: 배율(2 or 3 or 4)
    """
    if scaler not in [2, 3, 4]:
        print(f"Invalid scaler value: {scaler}. Must be 2, 3 or 4.")
        return None
    if not cv2.cuda.getCudaEnabledDeviceCount():
        print("No CUDA-enabled GPU found.")
        return None

    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    try:
        sr.readModel(f'models/EDSR_x{scaler}.pb')
    except Exception as e:
        print(f"Error reading model: {e}")
        return None

    # gpu acceleration
    sr.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    sr.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    sr.setModel('edsr', scaler)

    try:
        result = sr.upsample(img)
    except Exception as e:
        print(f"Error during upscaling: {e}")
        return None
    global upscaled_fraction_num
    upscaled_fraction_num += 1
    return result

def upscale_large_img_helper(img, scaler):
    if scaler not in [2, 3, 4]:
        print(f"Invalid scaler value: {scaler}. Must be 2, 3 or 4.")
        return None
    if not cv2.cuda.getCudaEnabledDeviceCount():
        print("No CUDA-enabled GPU found.")
        return None
    
    # TODO: 이미지 분할해서 각각 edsr 적용 후 합치기
    h, w, c = img.shape
    if h * w < PIXELS_LIMIT:
        return upscale_img(img, scaler)
    
    if w < h:
        upper = np.zeros((scaler*h, scaler*w, c))
        below = np.zeros((scaler*h, scaler*w, c))

        upper[0:scaler*(h//2 + OVERLAP_HALF_LENGTH),:] = upscale_large_img_helper(img[0:(h//2 + OVERLAP_HALF_LENGTH),:], scaler=scaler)
        below[scaler*(h//2 - OVERLAP_HALF_LENGTH):scaler*h,:] = upscale_large_img_helper(img[(h//2 - OVERLAP_HALF_LENGTH):h,:], scaler=scaler)

        # h//2 - OVERLAP_HALF_LENGTH ~ h//2 + OVERLAP_HALF_LENGTH
        alpha = np.ones((scaler*h, scaler*w)) * np.arange(scaler * h).reshape(-1, 1)
        start = scaler * (h//2 - OVERLAP_HALF_LENGTH)
        end = scaler * (h//2 + OVERLAP_HALF_LENGTH)
        alpha = np.clip((alpha - start) / (end - start), 0, 1)

        alpha_3ch = np.repeat(alpha[:, :, np.newaxis], 3, axis=2)

        upper_f = upper.astype(np.float32)
        below_f = below.astype(np.float32)

        blended = upper_f * (1-alpha_3ch) + below_f * alpha_3ch
        blended = np.clip(blended, 0, 255).astype(np.uint8)
        return blended
    
    else:
        left = np.zeros((scaler*h, scaler*w, c))
        right = np.zeros((scaler*h, scaler*w, c))
        
        left[:,0:scaler*(w//2 + OVERLAP_HALF_LENGTH)] = upscale_large_img_helper(img[:,0:(w//2 + OVERLAP_HALF_LENGTH)], scaler=scaler)
        right[:,scaler*(w//2 - OVERLAP_HALF_LENGTH):scaler*w] = upscale_large_img_helper(img[:,(w//2 - OVERLAP_HALF_LENGTH):w], scaler=scaler)

        # w//2 - OVERLAP_HALF_LENGTH ~ w//2 + OVERLAP_HALF_LENGTH
        alpha = np.ones((scaler*h, scaler*w)) * np.arange(scaler * w).reshape(1, -1)
        start = scaler * (w//2 - OVERLAP_HALF_LENGTH)
        end = scaler * (w//2 + OVERLAP_HALF_LENGTH)
        alpha = np.clip((alpha - start) / (end - start), 0, 1)
        
        alpha_3ch = np.repeat(alpha[:, :, np.newaxis], 3, axis=2)
        
        left_f = left.astype(np.float32)
        right_f = right.astype(np.float32)

        blended = left_f * (1-alpha_3ch) + right_f * alpha_3ch
        blended = np.clip(blended, 0, 255).astype(np.uint8)
        return blended

def upscale_large_img(img, scaler):
    global upscaled_fraction_num
    upscaled_fraction_num = 0

    result = upscale_large_img_helper(img, scaler=scaler)
    print(f'upscaled after being divided into {upscaled_fraction_num} fragments.')
    upscaled_fraction_num = 0
    return result

img = cv2.imread(lenna_path)
img2 = mr.upscale_large_img(img, scaler=4)

upscaled without fraction
8.416339635848999 sec taken


True

In [17]:

a = mr._upscale_by_edsr(lenna_path, 4)


5.010789394378662 sec taken


In [2]:
lenna_img = cv2.imread(lenna_path)

b = mr.upscale_large_img(lenna_img, 4)
c = mr.upscale_large_img(b, 4)


upscaled without fraction
7.298156976699829 sec taken qqqqq
upscaled after being divided into 32 fragments.
67.01304268836975 sec taken qqqqq


In [3]:
imshow(lenna_img)
imshow(b)
imshow(c)

Dropped Escape call with ulEscapeCode : 0x03007703


In [9]:
for _ in range(20):
    imshow(c)

In [30]:
print(np.arange(5))

[0 1 2 3 4]


In [47]:
img = cv2.imread(lenna_path)
print(img.shape)
ztest = np.zeros(img.shape)
print(ztest.shape)

(512, 512, 3)
(512, 512, 3)


In [2]:
cv2.getNumThreads()

16

In [3]:
os.cpu_count()

16

In [6]:
os.environ

environ{'USER': 'primarina314',
        'VSCODE_WSL_EXT_LOCATION': '/mnt/c/Users/user/.vscode/extensions/ms-vscode-remote.remote-wsl-0.99.0',
        'SHLVL': '2',
        'HOME': '/home/primarina314',
        'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus',
        'WSL_DISTRO_NAME': 'Ubuntu-22.04',
        'WAYLAND_DISPLAY': 'wayland-0',
        'LOGNAME': 'primarina314',
        'NAME': 'Code',
        'WSL_INTEROP': '/run/WSL/419_interop',
        'PULSE_SERVER': 'unix:/mnt/wslg/PulseServer',
        '_': '/home/primarina314/anaconda3/envs/mrs3/bin/python',
        'TERM': 'xterm-color',
        'PATH': '/home/primarina314/anaconda3/envs/mrs3/bin:/home/primarina314/.vscode-server/bin/19e0f9e681ecb8e5c09d8784acaa601316ca4571/bin/remote-cli:/home/primarina314/anaconda3/condabin:/usr/local/cuda-12.2/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/usr/lib/wsl/lib:/mnt/c/WINDOWS/system32:/mnt/c/WINDOWS:/mnt/c/WINDOWS/System32/Wbem